In [246]:
import numpy as np
import pandas as pd
import math
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from statistics import mode
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly"
input_file_name = "../datasets/ID_data_mass_18122012_cleared.xlsx"

In [247]:
df = pd.read_excel(input_file_name)
df

,номер,дата,Глубина манометра,Dшт,Руст,Рзаб,Pлин,Руст1,Рзаб2,Pлин3,...,Рпл. Тек (Расчет по КВД),Рпл. Тек (Карноухов),Pсб,Pсб5,Ro_g,Ro_c,Ro_w,Удельная плотность газа,G_total,КГФ
0,804,2008-06-05,3576.3,7.94,249.6,370.1000,101.800000,249.0,359.6,101.800000,...,56.5,NaN,93.6,92.376018,0.806017,801.0,1000.0,0.669449,2.782623,0.311909
1,804,2008-06-06,3576.3,9.53,233.5,364.6000,101.300000,231.0,338.1,102.400000,...,56.5,NaN,92.9,91.685171,0.806017,801.0,1000.0,0.669449,3.697781,0.288600
2,804,2008-06-07,3576.3,11.11,213.4,357.1000,101.600000,211.0,314.8,100.600000,...,56.5,NaN,91.4,90.204787,0.806017,801.0,1000.0,0.669449,4.515073,0.248791
3,804,2008-06-08,3576.3,12.70,191.6,347.4000,98.400000,187.0,291.5,99.000000,...,56.5,NaN,89.2,88.033555,0.806017,801.0,1000.0,0.669449,5.217673,0.223559
4,804,2008-06-09,3576.3,14.29,171.9,337.7000,99.200000,169.0,270.3,99.500000,...,56.5,NaN,89.7,88.527017,0.806017,801.0,1000.0,0.669449,5.765092,0.215149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,22505,2009-09-23,3678.0,12.70,182.2,271.5860,93.255111,179.0,258.6,92.702141,...,60.9,60.8,89.7,88.527017,NaN,NaN,NaN,NaN,NaN,NaN
181,22505,2009-09-27,3678.0,14.29,157.0,249.0760,96.472143,153.0,233.5,100.060417,...,60.9,60.8,87.3,86.158401,NaN,NaN,NaN,NaN,NaN,NaN
182,22505,2010-04-09,3679.5,7.94,211.9,289.3877,91.131226,208.0,289.6,91.304642,...,64.1,61.3,88.2,87.046632,NaN,NaN,NaN,NaN,NaN,NaN
183,22505,2010-04-14,3679.5,9.53,204.0,282.8709,91.173008,201.0,275.3,91.616821,...,64.1,61.3,88.5,87.342709,NaN,NaN,NaN,NaN,NaN,NaN


In [248]:
df.dtypes

номер                                      int64
дата                              datetime64[ns]
Глубина манометра                        float64
Dшт                                      float64
Руст                                     float64
Рзаб                                     float64
Pлин                                     float64
Руст1                                    float64
Рзаб2                                    float64
Pлин3                                    float64
Туст                                     float64
Тна шлейфе                               float64
Тзаб                                     float64
Tлин                                     float64
Дебит газа                               float64
Дебит ст. конд.                          float64
Дебит воды                               float64
Дебит смеси                              float64
Дебит гааз                               float64
Дебит кон нестабильный                   float64
Дебит воды4         

### Get statistic data

In [249]:
dropped_columns = ['номер', 'дата']
check_columns = df.drop(dropped_columns, axis=1).columns

In [250]:
class DataStatistic(object):
    # statistic init method: gets np_array data and returns Statistic object for it
    def __init__(self, data, name: str):
        # remove Nan items
        drop_nan_data = data[~np.isnan(data)]

        self.data = data
        self.size = np.size(data)            # all value's count
        self.value_count = np.size(drop_nan_data)    # not nan value's count
        self.average = np.mean(drop_nan_data)        # average value
        self.median = np.median(drop_nan_data)       # median
        self.mode = mode(drop_nan_data)
        self.quantile_first = np.quantile(drop_nan_data, 0.25)   # quantile of level 0.25
        self.quantile_third = np.quantile(drop_nan_data, 0.75)   # quantile of level 0.75
        self.cardinality = len(np.unique(drop_nan_data))         # cardinality - count of unique values
        self.min = np.min(drop_nan_data)     # min value
        self.max = np.max(drop_nan_data)     # max value
        self.skip_count = np.size(data[np.isnan(data)])  # count of empty values
        self.skip_percent = 100 * float(self.skip_count) / self.size   # percent of empty values
        self.name = name

    def print_statistic(self):
        print(self.name + " statistic parameters: ")
        # iterate over object attributes and print their values
        for attr, value in self.__dict__.items():
            if attr != 'name' and attr != 'data':
                print(attr + " : " + str(value))

# attributes statistic dict
statistic_data = dict()

for attribute in check_columns:
    data = df[attribute].to_numpy()
    statistic_data[attribute] = DataStatistic(name=attribute, data=data)

statistic_data["КГФ"].print_statistic()     # statistic data of all attributes

КГФ statistic parameters: 
size : 185
value_count : 93
average : 0.20170596768083773
median : 0.192
mode : 0.18
quantile_first : 0.164
quantile_third : 0.224
cardinality : 72
min : 0.131
max : 0.385415949386802
skip_count : 92
skip_percent : 49.729729729729726


In [251]:
base_dir = '../plots/lab1/'

# plot statistic data:
def add_line(fig, y_value, st_x, end_x, value_name: str, row, col):
    fig.add_trace(go.Scatter(x=[st_x, end_x], y=[y_value, y_value], name=value_name), row=row, col=col)

def add_statistic_description(fig, statistic: DataStatistic, row, col):
    attr_names = []
    attr_values = []
    # get list of attr names and list of attr values
    # (NOTE: save matching between items from each list!!!)
    for attr, value in statistic.__dict__.items():
        if attr != 'data':
            attr_names.append(attr)
            attr_values.append(value)
    # create table
    statistic_table = go.Table(header=dict(values=["statistic item", "value"]),
                               cells=dict(values=[attr_names, attr_values]))
    # add table to graphic
    fig.add_trace(statistic_table, row=row, col=col)

def plot_statistic_data(statistic: DataStatistic, attribute_name: str, attribute_data):
    # graphic contains statistic data and distribution
    graphic = make_subplots(rows=2, cols=1, specs=[[{"type":"xy"}],
                                                   [{"type":"table"}]] )
    # init distribution:
    value_count = len(attribute_data)
    x = np.linspace(1, value_count, value_count)
    fig = go.Scatter(x=x, y=attribute_data, name=attribute_name, mode='markers')  # plot distribution graphic as go.Scatter

    # add distribution to graphic
    graphic.add_trace(fig, row=1, col=1)
    # add quantile's lines
    add_line(graphic, statistic.quantile_first, 1, value_count, "Quantile first", row=1, col=1)
    add_line(graphic, statistic.quantile_third, 1, value_count, "Quantile third", row=1, col=1)

    # add statistic data to graphic
    add_statistic_description(graphic, statistic, row=2, col=1)
    # customize graphic - dimensions, title and other parameters
    graphic.update_layout(height=870, title_text="Attribute: " + attribute_name)

    # return result graphic
    return graphic

graphics = dict()

for attribute in check_columns:
    attribute_data = df[attribute].to_numpy()
    graphics[attribute] = plot_statistic_data(statistic_data[attribute], attribute, attribute_data)

for attribute in check_columns:
    file_name = base_dir + attribute.replace(".", "_")+ ".pdf"
    # graphics[attribute].show()
    graphics[attribute].write_image(file_name)

graphics['КГФ'].show()


### Check outliers

In [252]:
### return list of outliers values for the specified attribute
def get_attribute_outliers(statistic: DataStatistic):
    result = []

    q_1 = statistic.quantile_first
    q_3 = statistic.quantile_third
    data = statistic.data

    low_bound = q_1 - 1.5 * (q_3 - q_1)
    up_bound = q_3 + 1.5 * (q_3 - q_1)

    for i in range(len(data)):
        if not np.isnan(data[i]):
            if data[i] < low_bound or data[i] > up_bound:
                result.append(data[i])

    return result


# check categoricality for the specified attribute
def is_categorical(statistic: DataStatistic):
    cardinality_share_threshold = 0.15

    card = statistic.cardinality
    value_count = statistic.value_count

    return card / value_count <= cardinality_share_threshold


### just to look up at categorical attributes
def get_categorical_attributes(attributes_statistic_dict):
    result = []

    for attr in attributes_statistic_dict:
        if is_categorical(attributes_statistic_dict[attr]):
            result.append(attr)

    return result


### return data with supplemented skips
def supplement_attribute_skips(statistic: DataStatistic):
    attr_data = statistic.data

    if statistic.skip_percent <= 30:
        if is_categorical(statistic):
            supplement_val = statistic.mode
        else:
            supplement_val = statistic.median
        for i in range(len(attr_data)):
            if np.isnan(attr_data[i]):
                attr_data[i] = supplement_val

    return attr_data


### supplement data in dataframe
### input - cleared from outliers dataframe and statistic data and attributes
### list which are rejected for the supplementing
def supplement_dataframe_skips(cleared_df, statistic_data_dict, reject_supplement: list):
    attributes = cleared_df.columns

    for attr in attributes:
        if attr not in reject_supplement:
            cleared_df[attr] = supplement_attribute_skips(statistic_data_dict[attr])


### change outliers values to nan in the specified attribute
def clear_outliers(statistic: DataStatistic):
    q_1 = statistic.quantile_first
    q_3 = statistic.quantile_third
    data = statistic.data

    low_bound = q_1 - 1.5 * (q_3 - q_1)
    up_bound = q_3 + 1.5 * (q_3 - q_1)

    for i in range(len(data)):
        if not np.isnan(data[i]):
            if data[i] < low_bound or data[i] > up_bound:
                data[i] = np.nan
    return data


### returns dict of pairs: attribute-outliers list
def get_outliers(attributes_statistic_dict):
    result = dict()

    for attr in attributes_statistic_dict:
        result[attr] = get_attribute_outliers(attributes_statistic_dict[attr])

    return result

outliers_dict = get_outliers(statistic_data)
outliers_dict

{'Глубина манометра': [3952.0,
  3952.0,
  3952.0,
  3277.8,
  3277.8,
  3277.8,
  3277.8,
  3277.8,
  3276.0,
  3276.0,
  3276.0,
  3276.0,
  3276.0,
  3284.0,
  3284.0,
  3284.0,
  3284.0,
  3284.0,
  3288.6,
  3288.6,
  3289.8],
 'Dшт': [17.46],
 'Руст': [],
 'Рзаб': [181.13, 169.25, 179.555, 174.0],
 'Pлин': [79.95274967741933,
  80.73293237250546,
  73.30663802559425,
  76.5720676470589,
  121.6829297205758,
  19.6,
  19.83801298113208,
  34.16266740310079,
  35.39023160784315],
 'Руст1': [109.0],
 'Рзаб2': [169.2, 162.0, 160.0, 174.0],
 'Pлин3': [79.54112903225814,
  72.57925806451617,
  76.81951612903225,
  123.6168,
  40.73983870967736,
  45.06035483870962,
  47.84654838709679],
 'Туст': [24.2],
 'Тна шлейфе': [90.3, 95.5, 103.6, 106.8, 110.2],
 'Тзаб': [100.21],
 'Tлин': [],
 'Дебит газа': [],
 'Дебит ст. конд.': [272.7696100917432,
  302.6210767872898,
  262.0141659070192,
  248.0552876936173,
  284.2769335283365,
  264.3869160056256,
  264.0,
  293.863527702038,
  253.089064

In [253]:
attributes_with_outliers = []

# print attributes with outliers
for key in outliers_dict:
    if len(outliers_dict[key]) > 0:
        attributes_with_outliers.append(key)
        print(key)

Глубина манометра
Dшт
Рзаб
Pлин
Руст1
Рзаб2
Pлин3
Туст
Тна шлейфе
Тзаб
Дебит ст. конд.
Дебит воды
Дебит смеси
Дебит гааз
Дебит кон нестабильный
Дебит воды4
Рпл. Тек (послед точка на КВД)
Рпл. Тек (Расчет по КВД)
Ro_c
Удельная плотность газа 
G_total
КГФ


### Clear outliers

In [254]:
def remove_items(arr, removed_items):
    for item in removed_items:
        arr.remove(item)

reject_clear_outliers = ['Глубина манометра']
remove_items(attributes_with_outliers, reject_clear_outliers)

In [255]:
# clear outliers from dataframe and update statistic data
for attr in attributes_with_outliers:
    df[attr] = clear_outliers(statistic_data[attr])
    statistic_data[attr] = DataStatistic(data=df[attr].to_numpy(), name=attr)


### Supplement skips

In [256]:
### check categorical attributes
categorical_attributes = get_categorical_attributes(statistic_data)
categorical_attributes

['Dшт', 'Нэф', 'Ro_w']

In [257]:
### check categorical attribute before supplement
df[categorical_attributes[0]]

0       7.94
1       9.53
2      11.11
3      12.70
4      14.29
       ...  
180    12.70
181    14.29
182     7.94
183     9.53
184    11.11
Name: Dшт, Length: 185, dtype: float64

In [258]:
### check regular attribute before supplement
df['Pсб']

0      93.6
1      92.9
2      91.4
3      89.2
4      89.7
       ... 
180    89.7
181    87.3
182    88.2
183    88.5
184    89.8
Name: Pсб, Length: 185, dtype: float64

In [259]:
### supplement data
reject_supplement = ['дата', 'номер']
supplement_dataframe_skips(cleared_df=df, statistic_data_dict=statistic_data, reject_supplement=reject_supplement)
df

,номер,дата,Глубина манометра,Dшт,Руст,Рзаб,Pлин,Руст1,Рзаб2,Pлин3,...,Рпл. Тек (Расчет по КВД),Рпл. Тек (Карноухов),Pсб,Pсб5,Ro_g,Ro_c,Ro_w,Удельная плотность газа,G_total,КГФ
0,804,2008-06-05,3576.3,7.94,249.6,370.1000,101.800000,249.0,359.6,101.800000,...,56.5,50.7,93.6,92.376018,0.806017,801.0,1000.0,0.669449,2.782623,0.311909
1,804,2008-06-06,3576.3,9.53,233.5,364.6000,101.300000,231.0,338.1,102.400000,...,56.5,50.7,92.9,91.685171,0.806017,801.0,1000.0,0.669449,3.697781,0.288600
2,804,2008-06-07,3576.3,11.11,213.4,357.1000,101.600000,211.0,314.8,100.600000,...,56.5,50.7,91.4,90.204787,0.806017,801.0,1000.0,0.669449,4.515073,0.248791
3,804,2008-06-08,3576.3,12.70,191.6,347.4000,98.400000,187.0,291.5,99.000000,...,56.5,50.7,89.2,88.033555,0.806017,801.0,1000.0,0.669449,5.217673,0.223559
4,804,2008-06-09,3576.3,14.29,171.9,337.7000,99.200000,169.0,270.3,99.500000,...,56.5,50.7,89.7,88.527017,0.806017,801.0,1000.0,0.669449,5.765092,0.215149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,22505,2009-09-23,3678.0,12.70,182.2,271.5860,93.255111,179.0,258.6,92.702141,...,60.9,60.8,89.7,88.527017,NaN,NaN,NaN,NaN,NaN,NaN
181,22505,2009-09-27,3678.0,14.29,157.0,249.0760,96.472143,153.0,233.5,100.060417,...,60.9,60.8,87.3,86.158401,NaN,NaN,NaN,NaN,NaN,NaN
182,22505,2010-04-09,3679.5,7.94,211.9,289.3877,91.131226,208.0,289.6,91.304642,...,64.1,61.3,88.2,87.046632,NaN,NaN,NaN,NaN,NaN,NaN
183,22505,2010-04-14,3679.5,9.53,204.0,282.8709,91.173008,201.0,275.3,91.616821,...,64.1,61.3,88.5,87.342709,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
### check categorical attribute after supplement
df[categorical_attributes[0]]

0       7.94
1       9.53
2      11.11
3      12.70
4      14.29
       ...  
180    12.70
181    14.29
182     7.94
183     9.53
184    11.11
Name: Dшт, Length: 185, dtype: float64

In [261]:
### check regular attribute after supplement
df['Pсб']

0      93.6
1      92.9
2      91.4
3      89.2
4      89.7
       ... 
180    89.7
181    87.3
182    88.2
183    88.5
184    89.8
Name: Pсб, Length: 185, dtype: float64

In [262]:
### update statistic data
dropped_columns = ['номер', 'дата']
check_columns = df.drop(dropped_columns, axis=1).columns
updated_statistic_data = dict()

for attr in check_columns:
    updated_statistic_data[attr] = DataStatistic(data=df[attr].to_numpy(), name=attr)

### Check attributes with huge number of skips

In [263]:
def get_huge_skip_count_attributes(statistic_dict: dict):
    result = []

    for attr in statistic_dict:
        if statistic_dict[attr].skip_percent >= 60:
            result.append(attr)
    return result

huge_skip_count_attributes = get_huge_skip_count_attributes(updated_statistic_data)
huge_skip_count_attributes

['Ro_g', 'Ro_w', 'G_total']

In [264]:
### remove dst attribute G_total
huge_skip_count_attributes.remove('G_total')
huge_skip_count_attributes

['Ro_g', 'Ro_w']

In [265]:
# remove attributes with huge number of skips from dataframe and from update_statistic_data
for item in huge_skip_count_attributes:
    updated_statistic_data.pop(item)

df.drop(huge_skip_count_attributes, axis=1, inplace=True)

# remove redundant attribute 'номер'
df.drop('номер', axis=1, inplace=True)

print(updated_statistic_data.keys())
df

dict_keys(['Глубина манометра', 'Dшт', 'Руст', 'Рзаб', 'Pлин', 'Руст1', 'Рзаб2', 'Pлин3', 'Туст', 'Тна шлейфе', 'Тзаб', 'Tлин', 'Дебит газа', 'Дебит ст. конд.', 'Дебит воды', 'Дебит смеси', 'Дебит гааз', 'Дебит кон нестабильный', 'Дебит воды4', 'Нэф', 'Рпл. Тек (послед точка на КВД)', 'Рпл. Тек (Расчет по КВД)', 'Рпл. Тек (Карноухов)', 'Pсб', 'Pсб5', 'Ro_c', 'Удельная плотность газа ', 'G_total', 'КГФ'])


,дата,Глубина манометра,Dшт,Руст,Рзаб,Pлин,Руст1,Рзаб2,Pлин3,Туст,...,Нэф,Рпл. Тек (послед точка на КВД),Рпл. Тек (Расчет по КВД),Рпл. Тек (Карноухов),Pсб,Pсб5,Ro_c,Удельная плотность газа,G_total,КГФ
0,2008-06-05,3576.3,7.94,249.6,370.1000,101.800000,249.0,359.6,101.800000,53.0,...,56.8000,45.25000,56.5,50.7,93.6,92.376018,801.0,0.669449,2.782623,0.311909
1,2008-06-06,3576.3,9.53,233.5,364.6000,101.300000,231.0,338.1,102.400000,58.8,...,56.8000,45.25000,56.5,50.7,92.9,91.685171,801.0,0.669449,3.697781,0.288600
2,2008-06-07,3576.3,11.11,213.4,357.1000,101.600000,211.0,314.8,100.600000,63.6,...,56.8000,45.25000,56.5,50.7,91.4,90.204787,801.0,0.669449,4.515073,0.248791
3,2008-06-08,3576.3,12.70,191.6,347.4000,98.400000,187.0,291.5,99.000000,64.7,...,56.8000,45.25000,56.5,50.7,89.2,88.033555,801.0,0.669449,5.217673,0.223559
4,2008-06-09,3576.3,14.29,171.9,337.7000,99.200000,169.0,270.3,99.500000,64.0,...,56.8000,45.25000,56.5,50.7,89.7,88.527017,801.0,0.669449,5.765092,0.215149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2009-09-23,3678.0,12.70,182.2,271.5860,93.255111,179.0,258.6,92.702141,55.6,...,46.7435,36.96000,60.9,60.8,89.7,88.527017,NaN,NaN,NaN,NaN
181,2009-09-27,3678.0,14.29,157.0,249.0760,96.472143,153.0,233.5,100.060417,56.7,...,46.7435,36.96000,60.9,60.8,87.3,86.158401,NaN,NaN,NaN,NaN
182,2010-04-09,3679.5,7.94,211.9,289.3877,91.131226,208.0,289.6,91.304642,47.0,...,46.7435,38.65317,64.1,61.3,88.2,87.046632,NaN,NaN,NaN,NaN
183,2010-04-14,3679.5,9.53,204.0,282.8709,91.173008,201.0,275.3,91.616821,48.9,...,46.7435,38.65317,64.1,61.3,88.5,87.342709,NaN,NaN,NaN,NaN


### Check attributes with cardinality equals to 1

In [266]:
pure_attributes = []

for attr in updated_statistic_data:
    if updated_statistic_data[attr].cardinality == 1:
        pure_attributes.append(attr)
pure_attributes

[]

### Plot correlation matrix

In [268]:
corr_df = df.corr().apply(np.abs)
fig = ff.create_annotated_heatmap(np.around(corr_df.to_numpy(), 5), x=list(corr_df.columns), y=list(corr_df.columns), colorscale='Viridis', showscale=True)
fig.update_layout(width=2200, height=2200, title_text='correlation')
fig.show()
fig.write_image(base_dir + "correlation_lab1.pdf")

C:\Users\Andrey\AppData\Local\Temp\ipykernel_19792\2824618636.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

